In [2]:
from azureml.core import Workspace, Experiment

#ws = Workspace.get(name="udacity-project")
ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-145393
Azure region: southcentralus
Subscription id: 9e65f93e-bdd8-437b-b1e8-0647cd6098f7
Resource group: aml-quickstarts-145393


In [3]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

compute_cluster_name = 'compute-cluster'
compute_cluster_config = AmlCompute.provisioning_configuration(vm_size = "Standard_DS2_V2", max_nodes=4)
compute_cluster_target=ComputeTarget.create(ws,compute_cluster_name,compute_cluster_config)
compute_cluster_target.wait_for_completion(show_output=True)

Creating....
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


# 1. Setting up and running the HyperDrive run

In [3]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
from azureml.train.hyperdrive import choice
import os

# Specify parameter sampler
# no clue what range of values to provide, here. 
# for C, as the default is 1, I assume a normal distribution with mu=1
# as for max_iter, I will first start with a range 50-150, and see how it goes. 
ps = RandomParameterSampling(
    {
        "--C": uniform(0.01 , 1.99),
        "--max_iter": choice(range(50,150,10))
    }
)

# Specify a Policy
policy = BanditPolicy(slack_factor=0.1, evaluation_interval=1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory=os.path.join("./"), compute_target=compute_cluster_target,entry_script="train.py")

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(estimator=est, 
                                    hyperparameter_sampling=ps, 
                                    policy=policy,
                                    primary_metric_name="Accuracy",
                                    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                    max_total_runs=100,
                                    max_concurrent_runs=4)

'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.
'enabled' is deprecated. Please use the azureml.core.runconfig.DockerConfiguration object with the 'use_docker' param instead.


In [4]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hyperdrive_run = exp.submit(hyperdrive_config,show_output=True)
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [8]:
import joblib
# Get your best run and save the model from that run.
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']

print(best_run)
print(best_run_metrics)

best_run.get_file_names()

model = best_run.register_model(model_name='HD_model', model_path='./outputs/model.pkl')

Run(Experiment: udacity-project,
Id: HD_27eca9f0-e796-403a-b027-f54f8a18119e_5,
Type: azureml.scriptrun,
Status: Completed)
{'Regularization Strength:': 1.1777921298523928, 'Max iterations:': 50, 'Accuracy': 0.9176024279210926}


# 2. Setting up and running the AutoML run

In [4]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
path = r"https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds = TabularDatasetFactory.from_delimited_files(path=path)

In [5]:
from train import clean_data

# Use the clean_data function to clean your data.
X, y = clean_data(ds)

In [14]:
# as per article https://docs.microsoft.com/en-us/azure/machine-learning/how-to-configure-auto-train#configure-your-experiment-settings, the use of arguments X & y in automl_config has been deprecated.  
# replacing X,y by a complete set 'data' -> using pandas .concat method to reconcatenante dataframes.  

import pandas as pd
data = pd.concat([X,y], axis=1)
assert X.shape[1]+1==data.shape[1]


In [17]:
# got error:  "message": "Input of type '<class 'pandas.core.frame.DataFrame'>' is not supported. Supported types: [azureml.data.tabular_dataset.TabularDataset]"
# dataframe needs to be converted back to a tabulardataset object.
from azureml.data import TabularDataset

data.to_csv('data.csv')

datastore = ws.get_default_datastore()
datastore.upload_files(['data.csv'])
TDS_data = TabularDatasetFactory.from_delimited_files([(datastore,'data.csv')])


Uploading an estimated of 1 files
Target already exists. Skipping upload for data.csv
Uploaded 0 files


In [20]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.

# Note JCF: n_cross_validations=5, to stick to a size of test set of 20%

automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task= 'classification',
    primary_metric= 'accuracy',
    training_data= TDS_data,
    label_column_name='y',
    n_cross_validations=5,
    compute_target=compute_cluster_target,
    )

In [21]:
# Submit your automl run
from azureml.core.experiment import Experiment
experiment = Experiment(ws, "automl_test_experiment")
run = experiment.submit(config=automl_config, show_output=True)

Submitting remote run.
No run_configuration provided, running on compute-cluster with default configuration
Running on remote compute: compute-cluster


Experiment,Id,Type,Status,Details Page,Docs Page
automl_test_experiment,AutoML_4042bad0-a119-4a52-a475-debb8ade34bb,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+---------------------------------+---------------------------------+--------------------------------------+
|Size of the smallest class       |Name/Label of the sm

## Details of best run

In [23]:
# Retrieve and save your best automl model.

best_run, model = run.get_output()

In [27]:
best_run

Experiment,Id,Type,Status,Details Page,Docs Page
automl_test_experiment,AutoML_4042bad0-a119-4a52-a475-debb8ade34bb_24,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [28]:
best_run.get_details()

{'runId': 'AutoML_4042bad0-a119-4a52-a475-debb8ade34bb_24',
 'target': 'compute-cluster',
 'status': 'Completed',
 'startTimeUtc': '2021-05-22T17:43:42.782936Z',
 'endTimeUtc': '2021-05-22T17:44:53.82465Z',
 'properties': {'runTemplate': 'automl_child',
  'pipeline_id': '__AutoML_Ensemble__',
  'pipeline_spec': '{"pipeline_id":"__AutoML_Ensemble__","objects":[{"module":"azureml.train.automl.ensemble","class_name":"Ensemble","spec_class":"sklearn","param_args":[],"param_kwargs":{"automl_settings":"{\'task_type\':\'classification\',\'primary_metric\':\'accuracy\',\'verbosity\':20,\'ensemble_iterations\':15,\'is_timeseries\':False,\'name\':\'automl_test_experiment\',\'compute_target\':\'compute-cluster\',\'subscription_id\':\'9e65f93e-bdd8-437b-b1e8-0647cd6098f7\',\'region\':\'southcentralus\',\'spark_service\':None}","ensemble_run_id":"AutoML_4042bad0-a119-4a52-a475-debb8ade34bb_24","experiment_name":"automl_test_experiment","workspace_name":"quick-starts-ws-145393","subscription_id":"9e

## Saving the fitted model

In [30]:
import joblib
joblib.dump(model,'automl_fitted.pckl')

['automl_fitted.pckl']

## Proof of cluster clearnup

In [33]:
compute_cluster_target.delete()

In [34]:
from azureml.widgets import RunDetails
RunDetails(run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…